# Import

In [1]:
!pip install mlflow
!pip install torch==1.8.1
!pip install pytorch-lightning
!pip install transformers

In [2]:
import sys
sys.path.append('../../')

import os
import sys
import logging
import argparse
from pathlib import Path
from ast import literal_eval
from collections import Counter
from typing import Any, Dict, Optional

In [3]:
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

import sagemaker
from sagemaker import get_execution_role
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger


import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix


import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc

import transformers
from transformers import (
    AdamW,
    AutoConfig,
    AutoModel,
    AutoModelForSequenceClassification,
    T5ForConditionalGeneration,
    T5Tokenizer,
    AutoTokenizer,
)
from transformers.optimization import (
    Adafactor,
    get_linear_schedule_with_warmup,
)

Local constants, regarding the data, MLFlow server, paths, etc..: use them

In [4]:
from deep.constants import *
from deep.utils import *

In [5]:
%load_ext autoreload
%autoreload 2

# Data

In [6]:
train_dataset = pd.read_csv(LATEST_DATA_PATH / "data_v0.5_train.csv")
val_dataset = pd.read_csv(LATEST_DATA_PATH / "data_v0.5_val.csv")
##
train_dataset["sectors"] = train_dataset["sectors"].apply(literal_eval)
val_dataset["sectors"] = val_dataset["sectors"].apply(literal_eval)
##
sector_set = set()
for sectors_i in train_dataset["sectors"]:
    sector_set.update(sectors_i)
sectorname_to_sectorid = {sector:i for i, sector in enumerate(list(sorted(sector_set)))}

## Sagemaker Prep

### Session

In [7]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = 'arn:aws:iam::961104659532:role/service-role/AmazonSageMaker-ExecutionRole-20210519T102514'

### Bucket upload

In [8]:
sample = False

if sample:
    train_dataset = train_dataset.sample(100)
    val_dataset = val_dataset.sample(100)
    
job_name = f"pytorch-{formatted_time()}-test"
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name

train_path = str(input_path / 'train.pickle')
val_path = str(input_path / 'val.pickle')


train_dataset.to_pickle(train_path, protocol=4)
val_dataset.to_pickle(val_path, protocol=4)

### Estimator Definition

In [9]:
instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

In [10]:
input_path

S3Path('s3://sagemaker-deep-experiments-dev/training/input_data/pytorch-2021-07-07-18-20-42-922-test')

In [11]:
from sagemaker.pytorch import PyTorch

hyperparameters={
    'tracking_uri': MLFLOW_SERVER,
    'experiment_name': 'pl_test',
    'max_len': 200,
    'epochs': 1,
    'train_batch_size': 16,
    'eval_batch_size': 16,
    'model_name': 'sentence-transformers/paraphrase-mpnet-base-v2',
    'classes': str(SECTORS)
}

estimator = PyTorch(
    entry_point='train.py',
    source_dir=str(SCRIPTS_EXAMPLES_PATH / 'sector-pl'),
    output_path=str(DEV_BUCKET / 'models/'),
    code_location=str(input_path),
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    role=role,
    framework_version='1.8',
    py_version='py36',
    hyperparameters = hyperparameters,
    job_name=job_name,
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
)

In [12]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [13]:
estimator.fit(fit_arguments, job_name=job_name)

2021-07-07 16:21:11 Starting - Starting the training job...
2021-07-07 16:21:34 Starting - Launching requested ML instancesProfilerReport-1625674868: InProgress
......
2021-07-07 16:22:34 Starting - Preparing the instances for training......
2021-07-07 16:23:54 Downloading - Downloading input data...
2021-07-07 16:24:14 Training - Downloading the training image.......................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-07-07 16:28:24,314 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-07-07 16:28:24,338 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-07-07 16:28:27,416 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-07-07 16:28:27,734 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt
  Crea